## CNN

In [53]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_DATA/', one_hot = True)
tf.reset_default_graph()

Extracting MNIST_DATA/train-images-idx3-ubyte.gz
Extracting MNIST_DATA/train-labels-idx1-ubyte.gz
Extracting MNIST_DATA/t10k-images-idx3-ubyte.gz
Extracting MNIST_DATA/t10k-labels-idx1-ubyte.gz


In [54]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [55]:
#conv layer 1(첫 번째 레이어고고)

X = tf.placeholder(tf.float32, [None, 784])
#None: n개의 이미지, 784개의 값
X_img = tf.reshape(X, [-1, 28, 28, 1])
#28X28, 1:색깔, -1:N개 (알아서 해라)
Y = tf.placeholder(tf.float32, [None, 10])

W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev = 0.01))
#3X3 filter, num_of_color = 1, num_of_lifters = 32
#32개의 convolution값들이 생겨난다.

print('<Convolution Layer1>\n')

L1 = tf.nn.conv2d(X_img, W1, strides = [1,1,1,1], padding = 'SAME')
print('convolution: ',L1)
'''padding = "SAME": 스트라이드가 1X1이 기준일 때 WEIGHT에 상관없이
L1의 출력값은 입력의 이미지 사이즈와 같다
CONVOLUTION을 통과해서 나오면 28X28 사이즈,
대신 필터를 32개를 사용했기 때문에 32개의 convolution값들이 생김.'''
L1 = tf.nn.relu(L1)
print('relu함수 적용: ',L1)
L1 = tf.nn.max_pool(L1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
print('pooling 적용: ',L1)
#padding = 2x2, stride가 1이면 28x28이지만, 2이기때문에 14x14가 나옴, 채널의 개수는 32개

#conv layer2(두번째 레이어 고고)
W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev = 0.01))
#pooling 적용:  Tensor("MaxPool_2:0", shape=(?, 14, 14, 32), dtype=float32)
#3x3 filter, 앞에서 32는 똑같이, 이번엔 64개의 필터로 설정하겠다.
#conv => (?, 14, 14, 64) 필터를 64개 사용했기 때문
#pool => (?, 7, 7, 64) 스트라이드를 2x2로 했기 때문에 7x7
print('\n<Convolution Layer2>\n')
L2 = tf.nn.conv2d(L1, W2, strides = [1,1,1,1], padding = 'SAME')
print('convolution: ', L2)
L2 = tf.nn.relu(L2)
print('relu 통과: ', L2)
L2 = tf.nn.max_pool(L2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
print('pooling: ', L2)
L2 = tf.reshape(L2, [-1, 7 * 7 * 64])
print('reshape: ', L2)
#입체인 l2를 reshape로 펼친다. 7*7*64로 reshape, 이게 n개만큼 있다.

<Convolution Layer1>

convolution:  Tensor("Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)
relu함수 적용:  Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)
pooling 적용:  Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)

<Convolution Layer2>

convolution:  Tensor("Conv2D_1:0", shape=(?, 14, 14, 64), dtype=float32)
relu 통과:  Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)
pooling:  Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)
reshape:  Tensor("Reshape_1:0", shape=(?, 3136), dtype=float32)


### Fully connected layer

In [56]:
W3 = tf.get_variable("W3", shape = [7 * 7 * 64, 10], 
                     initializer = tf.contrib.layers.xavier_initializer())
#output:10개(0-9까지)

b = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2, W3) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = hypothesis, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost, optimizer], feed_dict = {X: batch_xs, Y: batch_ys})
        avg_cost += c / total_batch
    print('Epoch: ', '%04d' % (epoch + 1), 'cost = ', '{:.9f}'.format(avg_cost))
    
print('Learning Finished!')

correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy: ', sess.run(accuracy, feed_dict = {X: mnist.test.images, Y: mnist.test.labels}))


Learning started. It takes sometime.
Epoch:  0001 cost =  0.401931533
Epoch:  0002 cost =  0.106304011
Epoch:  0003 cost =  0.078407679
Epoch:  0004 cost =  0.064410166
Epoch:  0005 cost =  0.055115400
Epoch:  0006 cost =  0.047130910
Epoch:  0007 cost =  0.041969337
Epoch:  0008 cost =  0.036958858
Epoch:  0009 cost =  0.033934926
Epoch:  0010 cost =  0.029994835
Epoch:  0011 cost =  0.025108832
Epoch:  0012 cost =  0.024082376
Epoch:  0013 cost =  0.020188777
Epoch:  0014 cost =  0.019653976
Epoch:  0015 cost =  0.016476957
Learning Finished!
Accuracy:  0.9877


### Deep CNN

In [86]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_DATA/', one_hot = True)
tf.reset_default_graph()

Extracting MNIST_DATA/train-images-idx3-ubyte.gz
Extracting MNIST_DATA/train-labels-idx1-ubyte.gz
Extracting MNIST_DATA/t10k-images-idx3-ubyte.gz
Extracting MNIST_DATA/t10k-labels-idx1-ubyte.gz


In [87]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100
keep_prob = tf.placeholder(tf.float32)

In [88]:
X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev = 0.01))
#    Conv     -> (?, 28, 28, 32)
#    Pool     -> (?, 14, 14, 32)

L1 = tf.nn.conv2d(X_img, W1, strides = [1,1,1,1], padding = 'SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
L1 = tf.nn.dropout(L1, keep_prob = keep_prob)

W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev = 0.01))
#    Conv      ->(?, 14, 14, 64)
#    Pool      ->(?, 7, 7, 64)

L2 = tf.nn.conv2d(L1, W2, strides = [1,1,1,1], padding = 'SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
L2 = tf.nn.dropout(L2, keep_prob = keep_prob)

W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev = 0.01))
#    Conv      ->(?, 7, 7, 128)
#    Pool      ->(?, 4, 4, 128)
#    Reshape   ->(?, 4 * 4 * 128) # Flatten them for FC

L3 = tf.nn.conv2d(L2, W3, strides = [1,1,1,1], padding = 'SAME')
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
L3 = tf.nn.dropout(L3, keep_prob = keep_prob)

L3 = tf.reshape(L3, [-1, 128 * 4 * 4])

W4 = tf.get_variable('W4', shape = [128 * 4 * 4, 625],
                    initializer = tf.contrib.layers.xavier_initializer())
#625개 출력하겠습니다.

b4 = tf.Variable(tf.random_normal([625]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob = keep_prob)

W5 = tf.get_variable('W5', shape = [625, 10],
                    initializer = tf.contrib.layers.xavier_initializer())

b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4, W5) + b5

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits = hypothesis, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate  = learning_rate).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    print('Learning started. It takes some time.')

    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size)
        
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            c, _ = sess.run([cost, optimizer], feed_dict = {X: batch_xs, Y: batch_ys, keep_prob : 0.7})
            avg_cost += c / total_batch
        
        print('Epoch: ', '%04d' % (epoch + 1), 'cost = ', '{:.9f}'.format(avg_cost))
        
    print('Learning Finished!')

    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print('Accuracy: ', sess.run(accuracy, feed_dict= {
        X: mnist.test.images, Y: mnist.test.labels, keep_prob : 1}))

Learning started. It takes some time.
Epoch:  0001 cost =  0.374056701
Epoch:  0002 cost =  0.096251808
Epoch:  0003 cost =  0.071885036
Epoch:  0004 cost =  0.058476976
Epoch:  0005 cost =  0.050731856
Epoch:  0006 cost =  0.046820657
Epoch:  0007 cost =  0.041802602
Epoch:  0008 cost =  0.039408791
Epoch:  0009 cost =  0.036448343
Epoch:  0010 cost =  0.033645915
Epoch:  0011 cost =  0.031104443
Epoch:  0012 cost =  0.031094212
Epoch:  0013 cost =  0.028411251
Epoch:  0014 cost =  0.028107258
Epoch:  0015 cost =  0.025813088
Learning Finished!
Accuracy:  0.993
